In [1]:
from keras.models import Sequential
from keras.layers.normalization import BatchNormalization
from keras.layers.convolutional import Conv2D
from keras.layers.convolutional import MaxPooling2D
from keras.layers.advanced_activations import ELU,Softmax
from keras.layers.core import Activation
from keras.layers.core import Flatten
from keras.layers.core import Dropout
from keras.layers.core import Dense
from keras import backend as K
from keras.optimizers import Adam
#from config import emotion_config as config
from keras.preprocessing.image import ImageDataGenerator
from keras.models import load_model
import argparse
import os
import matplotlib.pyplot as plt
from keras.models import load_model
from keras.callbacks import ModelCheckpoint
import pandas as pd
import numpy as np


Using TensorFlow backend.


In [2]:
class EmotionVGGNet:

	@staticmethod
	def build(width, height, depth, classes):
		model = Sequential()
		inputShape = (height,width,depth)
		chanDim = -1

		if K.image_data_format == "channels_first":
			inputShape = (depth,height,width)
			chanDim = 1
		model = Sequential([
			Conv2D(32,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(32,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Conv2D(64,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(64,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Conv2D(64,(3,3), padding="same",
			kernel_initializer="he_normal",input_shape=inputShape),
			ELU(),
			BatchNormalization(axis=chanDim),
			Conv2D(64,(3,3), kernel_initializer="he_normal",
				padding="same"),
			ELU(),
			BatchNormalization(axis=chanDim),
			MaxPooling2D(pool_size=(2,2)),
			Dropout(0,25),
			Flatten(),
			Dense(64,kernel_initializer="he_normal"),
			ELU(),
			BatchNormalization(),
			Dropout(0.25),
			Dense(64,kernel_initializer="he_normal"),
			ELU(),
			BatchNormalization(),
			Dropout(0.25),
			Dense(classes,kernel_initializer="he_normal"),
			Softmax()])
		return model



In [3]:
emotion_dict = {0:"Angry", 1:"Disgust", 2:"Fear", 3:"Happy", 4:"Sad", 5:"Surprise", 6:"Neutral"}
BASE_DIR = "dataset/kaggle_fer/fer2013/"
data = pd.read_csv(BASE_DIR+"fer2013.csv")
#data.head()

In [18]:
train_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "Training"]["pixels"].values]
train_pixels = np.array([i.reshape((48,48)) for i in train_pixels],dtype=np.uint8)
train_labels = np.array([int(j) for j in data[data.Usage == 'Training']["emotion"].values],dtype=np.uint8)
train_pixels.shape,train_labels.shape,train_pixels[0]

((28709, 48, 48), (28709,), array([[ 70,  80,  82, ...,  52,  43,  41],
        [ 65,  61,  58, ...,  56,  52,  44],
        [ 50,  43,  54, ...,  49,  56,  47],
        ...,
        [ 91,  65,  42, ...,  72,  56,  43],
        [ 77,  82,  79, ..., 105,  70,  46],
        [ 77,  72,  84, ..., 106, 109,  82]], dtype=uint8))

In [5]:
test_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "PrivateTest"]["pixels"].values]
test_pixels = np.array([i.reshape((48,48)) for i in test_pixels])
test_labels = np.array([int(j) for j in data[data.Usage == 'PrivateTest']["emotion"].values])

In [6]:
validation_pixels = [np.array([int(i) for i in j.split()]) for j in data[data.Usage == "PublicTest"]["pixels"].values]
validation_pixels = np.array([i.reshape((48,48)) for i in validation_pixels])
validation_labels = np.array([int(j) for j in data[data.Usage == 'PublicTest']["emotion"].values])

In [7]:
train = ImageDataGenerator(rotation_range=10, zoom_range=0.1,horizontal_flip=True, rescale=1 / 255.0, fill_mode="nearest")
test = ImageDataGenerator(rescale=1 / 255.0)
validation = ImageDataGenerator(rescale=1 / 255.0)

In [10]:
print("[INFO] compiling model...")
model = EmotionVGGNet.build(width=48, height=48, depth=1,classes=len(emotion_dict))
opt = Adam(lr=1e-3)
model.compile(loss="categorical_crossentropy", optimizer=opt,metrics=["accuracy"])
print("[INFO] compiled")

[INFO] compiling model...
[INFO] compiled


In [19]:
checkpointer = ModelCheckpoint(filepath='weights/EmotionVGGNet.hdf5', verbose=1, save_best_only=True)

#model.fit_generator(
#    train.flow_from_dataframe(data,x_col=
#)

model.fit_generator(
	train.flow(train_pixels,train_labels,batch_size=32),
	validation_data = validation.flow(validation_pixels,validation_labels),
	epochs = 15,
	callbacks = [checkpointer],
	verbose=1)

ValueError: ('Input data in `NumpyArrayIterator` should have rank 4. You passed an array with shape', (28709, 48, 48))

In [ ]:
model.save("EmotionVGGNet.hdf5")
model = load_model("EmotionVGGNet.hdf5")

loss, acc = model.evaluate_generatoor(
	test.flow(test_pixels,test_labels,color_mode="grayscale"))

print("[INFO] accuracy : {:.2f}".format(acc))
